# Imports

In [36]:
import os
import textwrap
from dotenv import load_dotenv
import pprint

# Langchain
from langchain.chains import GraphCypherQAChain
from langchain.prompts.prompt import PromptTemplate
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI

# Warning control
import warnings
warnings.filterwarnings("ignore")

In [37]:
# Load from environment
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# Global constants
VECTOR_INDEX_NAME = 'film_overview_index'
VECTOR_NODE_LABEL = 'Film'
VECTOR_SOURCE_PROPERTY = 'overview'
VECTOR_EMBEDDING_PROPERTY = 'overviewEmbedding'

In [39]:
kg = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database=NEO4J_DATABASE
)

# Utils

## Delete all data

In [69]:
cypher = """
MATCH (n)
DETACH DELETE n
"""
kg.query(cypher)

[]

## Check for amount of total nodes

In [70]:
cypher = """
  MATCH (n) 
  RETURN count(n) AS numberOfNodes
"""
print(kg.query(cypher))

cypher = """
  MATCH (f:Film) 
  RETURN count(f) AS numberOfFilms
  """
print(kg.query(cypher))

[{'numberOfNodes': 0}]
[{'numberOfFilms': 0}]


## Remove and show indices

In [74]:
# kg.query("DROP INDEX `keyword`")
kg.query("SHOW INDEXES")
# kg.refresh_schema()
# print(kg.schema)

[{'id': 0,
  'name': 'index_343aff4e',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'NODE',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': neo4j.time.DateTime(2024, 3, 20, 18, 18, 44, 278000000, tzinfo=<UTC>),
  'readCount': 38224},
 {'id': 1,
  'name': 'index_f7700477',
  'state': 'ONLINE',
  'populationPercent': 100.0,
  'type': 'LOOKUP',
  'entityType': 'RELATIONSHIP',
  'labelsOrTypes': None,
  'properties': None,
  'indexProvider': 'token-lookup-1.0',
  'owningConstraint': None,
  'lastRead': None,
  'readCount': 0}]

# Creating list of films from CSV

In [75]:
import csv

all_films = []

YEARS = [2022, 2023]

for year in YEARS:
  with open(f'./data/{year}_movie_collection_data.csv', mode='r') as csv_file:
      csv_reader = csv.DictReader(csv_file)
      for row in csv_reader: # each row will be a dictionary
        all_films.append(row)

In [76]:
all_films

[{'Title': 'The Ice Age Adventures of Buck Wild',
  'Runtime (minutes)': '82',
  'Language': 'English',
  'Overview': "The fearless one-eyed weasel Buck teams up with mischievous possum brothers Crash & Eddie as they head off on a new adventure into Buck's home: The Dinosaur World.",
  'Release Date': '2022-01-28',
  'Genre': 'Animation, Comedy, Adventure, Family',
  'Keywords': 'ice age, snow, spin off, polar adventure',
  'Actors': 'Vincent Tong, Aaron Harris, Simon Pegg, Justina Milagros Machado, Utkarsh Ambudkar',
  'Director': 'John C. Donkin',
  'Stream': 'Disney Plus ',
  'Buy': '',
  'Rent': '',
  'Production Companies': 'Bardel Entertainment, Walt Disney Pictures',
  'Website': 'https://www.disneyplus.com/movies/the-ice-age-adventures-of-buck-wild/43OJuKxe9b9Y'},
 {'Title': 'After Ever Happy',
  'Runtime (minutes)': '95',
  'Language': 'English',
  'Overview': "As a shocking truth about a couple's families emerges, the two lovers discover they are not so different from each ot

In [77]:
first_film = all_films[0]
print(all_films)

[{'Title': 'The Ice Age Adventures of Buck Wild', 'Runtime (minutes)': '82', 'Language': 'English', 'Overview': "The fearless one-eyed weasel Buck teams up with mischievous possum brothers Crash & Eddie as they head off on a new adventure into Buck's home: The Dinosaur World.", 'Release Date': '2022-01-28', 'Genre': 'Animation, Comedy, Adventure, Family', 'Keywords': 'ice age, snow, spin off, polar adventure', 'Actors': 'Vincent Tong, Aaron Harris, Simon Pegg, Justina Milagros Machado, Utkarsh Ambudkar', 'Director': 'John C. Donkin', 'Stream': 'Disney Plus ', 'Buy': '', 'Rent': '', 'Production Companies': 'Bardel Entertainment, Walt Disney Pictures', 'Website': 'https://www.disneyplus.com/movies/the-ice-age-adventures-of-buck-wild/43OJuKxe9b9Y'}, {'Title': 'After Ever Happy', 'Runtime (minutes)': '95', 'Language': 'English', 'Overview': "As a shocking truth about a couple's families emerges, the two lovers discover they are not so different from each other. Tessa is no longer the sweet

# Create Film and Genre Node, with Relationship

In [78]:
# first_films = all_films[0:5]

for film in all_films:
    genre_list = film['Genre'].split(", ")
    actor_list = film['Actors'].split(", ")
    prod_comp_list = film['Production Companies'].split(", ")

    cypher_query = """
    MERGE (film:Film {title: $title})
    ON CREATE
        SET film.runtime = toInteger($runtime),
            film.language = $language,
            film.overview = $overview,
            film.release_date = date($release_date),
            film.keywords = $keywords,
            film.source = $website
    WITH film
    UNWIND $genres AS genre_type
    UNWIND $actors AS actor_name
    UNWIND $prodcution_companies AS company_name

    MERGE (genre:Genre {type: genre_type})
    MERGE (actor:Actor {name: actor_name})
    MERGE (director:Director {name: $director})
    MERGE (company:Production_Company {name: company_name})

    MERGE (film)-[:HAS_GENRE]->(genre)
    MERGE (director)-[:HAS_DIRECTED]->(film)
    MERGE (actor)-[:STARRED_IN]->(film)
    MERGE (company)-[:PRODUCED]->(film)

    """

    kg.query(cypher_query, params={
        'title': film['Title'],
        'runtime': film['Runtime (minutes)'],
        'language': film['Language'],
        'overview': film['Overview'],
        'release_date': film['Release Date'],
        'keywords': film['Keywords'],
        'website': film['Website'],
        'genres': genre_list,
        'actors': actor_list,
        'director': film['Director'],
        'prodcution_companies': prod_comp_list
    })

In [79]:
kg.refresh_schema()
print(textwrap.fill(kg.schema, 60))

Node properties are the following: Film {title: STRING,
runtime: INTEGER, language: STRING, overview: STRING,
release_date: DATE, keywords: STRING, source: STRING},Genre
{type: STRING},Actor {name: STRING},Director {name:
STRING},Production_Company {name: STRING} Relationship
properties are the following:  The relationships are the
following: (:Film)-[:HAS_GENRE]->(:Genre),(:Actor)-
[:STARRED_IN]->(:Film),(:Director)-[:HAS_DIRECTED]-
>(:Film),(:Production_Company)-[:PRODUCED]->(:Film)


# Sample Queries

## Getting all actors in a film

In [280]:
cypher = """
  MATCH (a:Actor)-[r:STARRED_IN]->(f:Film)
    WHERE f.title = $title
  RETURN a.name
"""

actors_info = kg.query(cypher, params={
    'title': 'Jurassic World Dominion'
})

for item in actors_info:
    print(item)

{'a.name': 'Laura Dern'}
{'a.name': 'Bryce Dallas Howard'}
{'a.name': 'Sam Neill'}
{'a.name': 'Chris Pratt'}
{'a.name': 'Jeff Goldblum'}


## Getting all films in a genre

In [281]:
cypher = """
  MATCH (f:Film)-[r:HAS_GENRE]->(g:Genre)
    WHERE g.type = $genre
  RETURN f.title
"""

films_in_genre = kg.query(cypher, params={
    'genre': 'Horror'
})

for item in films_in_genre:
    print(item)

{'f.title': 'Reign of Chaos'}
{'f.title': 'The OctoGames'}
{'f.title': 'Before Night Falls'}
{'f.title': 'The One Hundred'}
{'f.title': 'Beast'}
{'f.title': 'Halloween Ends'}
{'f.title': 'The Exorcism of God'}
{'f.title': 'Slash/Back'}
{'f.title': 'Nope'}
{'f.title': 'Witch Trials'}
{'f.title': 'Hellraiser'}
{'f.title': 'Scream'}
{'f.title': 'Project Wolf Hunting'}
{'f.title': 'Orphan: First Kill'}
{'f.title': 'Venus'}
{'f.title': 'Wyrmwood: Apocalypse'}
{'f.title': 'Smile'}
{'f.title': 'Pearl'}
{'f.title': 'The Black Phone'}
{'f.title': 'M3GAN'}
{'f.title': 'Jeepers Creepers: Reborn'}
{'f.title': 'The Exorcism of Hannah Stevenson'}
{'f.title': 'Good Boy'}
{'f.title': 'X'}
{'f.title': 'The Menu'}
{'f.title': 'Terrifier 2'}
{'f.title': 'Deep Fear'}
{'f.title': 'Fear'}
{'f.title': 'Godzilla Minus One'}
{'f.title': 'Scream VI'}
{'f.title': 'Sleep'}
{'f.title': "Five Nights at Freddy's"}
{'f.title': 'Squealer'}
{'f.title': 'Carousel'}
{'f.title': 'Saw X'}
{'f.title': 'Skal - Fight for Surv

# Get all films within a few nodes of a particular film (excluding via genre)

In [318]:
cypher = """
    MATCH (start:Film {title: $title})-[r:HAS_DIRECTED|STARRED_IN|PRODUCED*1..2]-(connected:Film)
    WITH start, connected, r, type(head(r)) AS related_by
    OPTIONAL MATCH (start)<-[:PRODUCED]-(p:Production_Company)-[:PRODUCED]->(connected)
    OPTIONAL MATCH (start)<-[:STARRED_IN]-(a:Actor)-[:STARRED_IN]->(connected)
    OPTIONAL MATCH (start)<-[:HAS_DIRECTED]-(d:Director)-[:HAS_DIRECTED]->(connected)
    RETURN connected.title as related_film, 
            related_by, 
            collect(distinct p.name) as production_companies, 
            collect(distinct a.name) as actors, 
            collect(distinct d.name) as directors
    """

related_films = kg.query(cypher, params={
    'title': 'The Menu'
})

pprint.pprint(related_films)

[{'actors': [],
  'directors': [],
  'production_companies': ['Searchlight Pictures', 'TSG Entertainment'],
  'related_by': 'PRODUCED',
  'related_film': 'Poor Things'},
 {'actors': [],
  'directors': [],
  'production_companies': ['TSG Entertainment'],
  'related_by': 'PRODUCED',
  'related_film': '65'},
 {'actors': ['Anya Taylor-Joy'],
  'directors': [],
  'production_companies': [],
  'related_by': 'STARRED_IN',
  'related_film': 'The Super Mario Bros. Movie'}]


## Get all films directed by the director of a particular film

In [283]:
# Getting all films directed by whoever directed 'The Next 365 Days'
cypher = """
    MATCH (start:Film {title: $title})<-[:HAS_DIRECTED]-(d:Director)-[:HAS_DIRECTED]->(connected:Film)
    RETURN connected.title
"""

related_films = kg.query(cypher, params={
    'title': 'The Next 365 Days'
})

for item in related_films:
    print(item)

# Getting directors who directed more than 1 film
cypher = """
MATCH (d:Director)-[:HAS_DIRECTED]->(f:Film)
WITH d, count(f) AS numFilms
WHERE numFilms > 1
RETURN d.name
"""

multifilm_directors = kg.query(cypher)
print(multifilm_directors)

{'connected.title': '365 Days: This Day'}
[{'d.name': 'Barbara Białowąs, Tomasz Mandes'}, {'d.name': 'Matt Bettinelli-Olpin, Tyler Gillett'}, {'d.name': 'Ti West'}]


## Getting Films (Title + Overview + Keywords) Directed by a Director

In [289]:
cypher = """
  MATCH (d:Director)-[r:HAS_DIRECTED]->(f:Film)
    WHERE d.name = $director
  RETURN f.title, f.overview, f.keywords
"""

director_films = kg.query(cypher, params={
    'director': 'Aaron Mirtes'
})

pprint.pprint(director_films)

[{'f.keywords': 'None',
  'f.overview': "Eight contestants compete in eight deadly, classic children's "
                'games. They seek fame beyond their wildest dreams, competing '
                'for the chance to take over the YouTube channel of the famous '
                'yet elusive masked content creator known only as "JaxPro".',
  'f.title': 'The OctoGames'}]


## Checking to see number of comedy films

In [284]:
cypher = """
  MATCH (f:Film)-[:HAS_GENRE]->(g:Genre {type: 'Comedy'})
  RETURN count(f) AS NumberOfComedyFilms
  """
kg.query(cypher)

[{'NumberOfComedyFilms': 47}]

## Getting all films produced by a production company

In [290]:
cypher = """
  MATCH (c:Production_Company)-[r:PRODUCED]->(f:Film)
    WHERE c.name = $name
  RETURN f.title, f.overview, f.keywords
"""

production_comp_films = kg.query(cypher, params={
    'name': 'A24'
})

pprint.pprint(production_comp_films)

[{'f.keywords': 'regret, nurse, missionary, idaho, bible, redemption, '
                'overweight man, addiction, based on play or musical, teacher, '
                'grief, neighbor, obesity, religion, death of lover, election, '
                'rebellious daughter, guilt, death, lgbt, sister-in-law, '
                'eating disorder, father daughter reunion, empathy, shame, '
                'english teacher, abandonment, one location, father daughter '
                'relationship, 2010s, gay theme, apartment, essay, food '
                'addiction, religious symbolism',
  'f.overview': 'A reclusive English teacher suffering from severe obesity '
                'attempts to reconnect with his estranged teenage daughter for '
                'one last chance at redemption.',
  'f.title': 'The Whale'},
 {'f.keywords': 'mother, martial arts, kung fu, philosophy, generations '
                'conflict, chinese woman, laundromat, chinese, east asian '
                'lead, div

## Get all horror films produced after a particular date

In [291]:
cypher = """
  MATCH (f:Film)-[:HAS_GENRE]->(g:Genre{type: $genre})
    WHERE f.release_date > date($date)
  RETURN f.title, f.overview, f.keywords
"""

new_horror = kg.query(cypher, params={
    'genre': 'Horror',
    'date': '2023-04-01'
})

pprint.pprint(new_horror)

[{'f.keywords': 'shark',
  'f.overview': 'A solo trip aboard a yacht takes a terrifying turn when a '
                'woman encounters three drug traffickers clinging to the '
                'shattered remains of a boat. They soon force her to dive into '
                'shark-infested waters to retrieve kilos of cocaine from the '
                'sunken wreck.',
  'f.title': 'Deep Fear'},
 {'f.keywords': 'monster, loss of loved one, giant monster, kamikaze, duty, '
                'atomic bomb test, radioactivity, naval combat, reboot, kaiju, '
                'duringcreditsstinger, post war japan, 1940s, naval battle, '
                'naval battleship, found family, godzilla, war orphan, '
                'survivor guilt, tokusatsu',
  'f.overview': 'Postwar Japan is at its lowest point when a new crisis '
                'emerges in the form of a giant monster, baptized in the '
                'horrific power of the atomic bomb.',
  'f.title': 'Godzilla Minus One'},
 {'f.keyw

# Use Similarity Search

In [28]:
retrieval_query_window = """
WITH node, score
RETURN "Title: " + node.title + "\n" + 
        "Overview: " + node.overview  + "\n" + 
        "Release Date: " + node.release_date  + "\n" + 
        "Runtime: " + node.runtime + " minutes" + "\n" + 
        "Language: " + node.language  + "\n" + 
        "Keywords: " + node.keywords  + "\n" + 
        "Source: " + node.source  + "\n" +
        "Score: " + score + "\n"
        as text,
    score,
    node {.source} AS metadata
"""

# vector_store_window = Neo4jVector.from_existing_index(
#     embedding=OpenAIEmbeddings(),
#     url=NEO4J_URI,
#     username=NEO4J_USERNAME,
#     password=NEO4J_PASSWORD,
#     database="neo4j",
#     index_name=VECTOR_INDEX_NAME,
#     text_node_property=VECTOR_SOURCE_PROPERTY,
#     retrieval_query=retrieval_query_window
# )

vector_store_window = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="film_index",
    node_label="Film",
    text_node_properties=["overview", "keywords"],
    embedding_node_property="embedding",
    search_type="hybrid",
    retrieval_query=retrieval_query_window
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever()

# Create a chatbot Question & Answer chain from the retriever
chain_window = RetrievalQAWithSourcesChain.from_chain_type(
    ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0, streaming=True),
    chain_type="stuff",
    retriever=retriever_window
)

In [29]:
question = "Recommend me some films that are similar to Wes Anderson films."

# answer = chain_window(
#     {"question": question},
#     return_only_outputs=True,
# )
# print(textwrap.fill(answer["answer"]))

for chunk in chain_window.stream(question):
    print(chunk, end="", flush=True)

Failed to write data to connection ResolvedIPv4Address(('34.66.78.163', 7687)) (ResolvedIPv4Address(('34.66.78.163', 7687)))
Failed to write data to connection IPv4Address(('4c38bf4f.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.66.78.163', 7687)))


{'question': 'Recommend me some films that are similar to Wes Anderson films.', 'answer': 'Some films similar to Wes Anderson films are "American Fiction," "Babylon," "The Fabelmans," and "Elemental."\n', 'sources': 'https://www.mgm.com/movies/american-fiction, https://www.babylonmovie.com/, https://www.thefabelmans.movie, https://movies.disney.com/elemental'}

# Using an LLM to write Cypher queries

In [336]:
CYPHER_GENERATION_TEMPLATE = """
Task:Generate a Cypher statement to query a graph database.
Instructions: Use only the provided relationship types and properties 
in the schema. Do not use any other relationship types or properties that 
are not provided.
Schema: {schema}
Note: Do not include any explanations or apologies in your responses.
Do not respond to any questions that might ask anything else than 
for you to construct a Cypher statement. Do not include any text except 
the generated Cypher statement. The output will be a list of python dictionaries.
Parse the output to get the answer to the question. For example, a response of
[{{'film.title': 'The OctoGames'}}] implies this is a film, with the title 'The
Octogames'.

Examples: Here are a few examples of generated Cypher 
statements for particular questions:

# Give me a list of all comedy films.
  MATCH (film:Film)-[r:HAS_GENRE]->(genre:Genre)
    WHERE genre.type = 'Comedy'
  RETURN film.title AS Comedy_Film

# What are all horror films produced after April 1st 2022?
MATCH (film:Film)-[:HAS_GENRE]->(genre:Genre{{type: 'Horror'}})
WHERE film.release_date > date('2023-04-01')
RETURN film.title as Horror_Film, film.release_date As Release_Date

# What are all films produced by A24?
MATCH (company:Production_Company)-[r:PRODUCED]->(film:Film)
WHERE company.name = 'A24'
RETURN film.title AS A24_Film

# Give me a list of all films associated with the movie 'Everything Everywhere All at Once'.
MATCH (start:Film {{title: 'Everything Everywhere All at Once'}})-[r:HAS_DIRECTED|STARRED_IN|PRODUCED*1..2]-(connected:Film)
WITH start, connected, r, type(head(r)) AS related_by
OPTIONAL MATCH (start)<-[:PRODUCED]-(p:Production_Company)-[:PRODUCED]->(connected)
OPTIONAL MATCH (start)<-[:STARRED_IN]-(a:Actor)-[:STARRED_IN]->(connected)
OPTIONAL MATCH (start)<-[:HAS_DIRECTED]-(d:Director)-[:HAS_DIRECTED]->(connected)
RETURN connected.title as Film_Associated_with_The_Menu, 
  related_by, 
  collect(distinct p.name) as production_companies, 
  collect(distinct a.name) as actors, 
  collect(distinct d.name) as directors
The question is:
{question}"""

CYPHER_GENERATION_PROMPT = PromptTemplate(
    input_variables=["schema", "question"], 
    template=CYPHER_GENERATION_TEMPLATE
)

chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0),
    graph=kg,
    verbose=True,
    cypher_prompt=CYPHER_GENERATION_PROMPT,
    validate_cypher=True,
)

In [337]:
def prettyCypherChain(question: str) -> str:
    response = chain.run(question)
    print(textwrap.fill(response, 90))
    return response

output = prettyCypherChain(
    "Give me a list of all films associated with the movie 'Everything Everywhere All at Once', along with how they are related.")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (start:Film {title: 'Everything Everywhere All at Once'})-[r:HAS_DIRECTED|STARRED_IN|PRODUCED*1..2]-(connected:Film)
WITH start, connected, r, type(head(r)) AS related_by
OPTIONAL MATCH (start)<-[:PRODUCED]-(p:Production_Company)-[:PRODUCED]->(connected)
OPTIONAL MATCH (start)<-[:STARRED_IN]-(a:Actor)-[:STARRED_IN]->(connected)
OPTIONAL MATCH (start)<-[:HAS_DIRECTED]-(d:Director)-[:HAS_DIRECTED]->(connected)
RETURN connected.title as Film_Associated_with_The_Menu, 
  related_by, 
  collect(distinct p.name) as production_companies, 
  collect(distinct a.name) as actors, 
  collect(distinct d.name) as directors
Full Context:
[{'Film_Associated_with_The_Menu': 'Past Lives', 'related_by': 'PRODUCED', 'production_companies': ['A24'], 'actors': [], 'directors': []}, {'Film_Associated_with_The_Menu': 'The Zone of Interest', 'related_by': 'PRODUCED', 'production_companies': ['A24'], 'actors': [], 'directors': []}, {'Film_Asso

# Agent

In [83]:
from langchain.agents import AgentExecutor
from langchain.tools.retriever import create_retriever_tool
from langchain.agents import create_openai_tools_agent
from langchain_core.prompts import ChatPromptTemplate
from langchain.prompts import HumanMessagePromptTemplate, MessagesPlaceholder

retrieval_query_window = """
MATCH (node:Film)-[:HAS_GENRE]->(genre:Genre)
MATCH (actor:Actor)-[:STARRED_IN]->(node)
MATCH (director:Director)-[:HAS_DIRECTED]->(node)
MATCH (prod_co:Production_Company)-[:PRODUCED]->(node)
WITH 
    node, collect(distinct genre.type) as genres, 
    collect(distinct actor.name) as actors, 
    collect(distinct director.name) as directors, 
    collect(distinct prod_co.name) as companies, 
    score
RETURN "Title: " + node.title + "\n" + 
        "Overview: " + node.overview  + "\n" + 
        "Release Date: " + node.release_date  + "\n" + 
        "Runtime: " + node.runtime + " minutes" + "\n" + 
        "Language: " + node.language  + "\n" + 
        "Keywords: " + node.keywords  + "\n" + 
        "Genres: " + apoc.text.join(genres, ', ') + "\n" +
        "Actors: " + apoc.text.join(actors, ', ') + "\n" +
        "Directors: " + apoc.text.join(directors, ', ') + "\n" +
        "Production Companies: " + apoc.text.join(companies, ', ') + "\n" +
        "Source: " + node.source  + "\n"
        as text, score, node {.source} AS metadata
"""

vector_store_window = Neo4jVector.from_existing_graph(
    embedding=OpenAIEmbeddings(),
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="film_index",
    node_label="Film",
    text_node_properties=["overview", "keywords", "language", "release_date", "runtime"],
    embedding_node_property="embedding",
    search_type="hybrid",
    retrieval_query=retrieval_query_window
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever()

# Create tool from retriever
retriever_tool = create_retriever_tool(
    retriever_window,
    "movie_data",
    """
    Use this tool to get films to recommend to the user.
    """
)

tools = [retriever_tool]
chat_model = ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0.5, streaming=True)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            'system',
            """
            Your goal is to recommend films to users based on their query and the retrieved context. 
            If a film doesn't seem relevant, omit it from your response. You should recommend no more than five films. 
            Your recommendation should be original, insightful, and at least two to three sentences long. Only recommend films 
            shown in your context. 

            # TEMPLATE FOR OUTPUT
            - [Title of Film]:
                - Runtime:
                - Release Date:
                - Recommendation: 
                - Source: 
            ...
             
            """            
        ),
        MessagesPlaceholder(variable_name='chat_history', optional=True),
        HumanMessagePromptTemplate(prompt=PromptTemplate(
            input_variables=['input'], template='{input}')),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent = create_openai_tools_agent(chat_model.with_config(
    {"tags": ["agent_llm"]}), tools, prompt_template)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True).with_config(
    {"run_name": "Agent"})

In [84]:
agent_executor.invoke(
    {"input": "Recommend me some films that are similar to Wes Anderson films."})



> Entering new AgentExecutor chain...

Invoking: `movie_data` with `{'query': 'Wes Anderson'}`


Title: Babylon
Overview: A tale of outsized ambition and outrageous excess, tracing the rise and fall of multiple characters in an era of unbridled decadence and depravity during Hollywood's transition from silent films to sound films in the late 1920s.
Release Date: 2022-12-22
Runtime: 189 minutes
Language: English
Keywords: elephant, gambling, gambling debt, movie business, orgy, cocaine, champagne, alcoholism, hollywood, alcoholic, filmmaking, movie star, silent film star, 1920s, old hollywood, fading star, sex
Genres: Comedy, Drama
Actors: Brad Pitt, Flea, Diego Calva, Jean Smart, Margot Robbie
Directors: Damien Chazelle
Production Companies: Paramount, Marc Platt Productions, Material Pictures, C2 Motion Picture Group, Wild Chickens, Organism Pictures
Source: https://www.babylonmovie.com/


Title: Fantastic Beasts: The Secrets of Dumbledore
Overview: Professor Albus Dumbledore knows 

{'input': 'Recommend me some films that are similar to Wes Anderson films.',
 'output': '- [Babylon]:\n    - Runtime: 189 minutes\n    - Release Date: 2022-12-22\n    - Recommendation: "Babylon" is a tale of outsized ambition and outrageous excess, similar to the eccentric and visually captivating storytelling style often found in Wes Anderson films. It explores the rise and fall of characters in a unique setting, mirroring Anderson\'s distinctive approach to storytelling.\n    - Source: [Official Website](https://www.babylonmovie.com/)\n\n- [Wonka]:\n    - Runtime: 117 minutes\n    - Release Date: 2023-12-06\n    - Recommendation: "Wonka" is a whimsical and imaginative film that captures the essence of creative storytelling, much like Wes Anderson\'s films. With its focus on dreams, chocolate, and the extraordinary, "Wonka" offers a similar charm and fantastical elements that Anderson fans may appreciate.\n    - Source: [Official Website](https://www.wonkamovie.com)'}

In [82]:
from chat_app import FilmSearch

agent = FilmSearch()

response = agent.ask(
    {"input": "Recommend me some Japanese horror films from 2023."})



> Entering new AgentExecutor chain...

Invoking: `movie_data` with `{'query': 'Japanese horror films 2023'}`


Title: Godzilla Minus One
Overview: Postwar Japan is at its lowest point when a new crisis emerges in the form of a giant monster, baptized in the horrific power of the atomic bomb.
Release Date: 2023-11-03
Runtime: 125 minutes
Language: Japanese
Keywords: monster, loss of loved one, giant monster, kamikaze, duty, atomic bomb test, radioactivity, naval combat, reboot, kaiju, duringcreditsstinger, post war japan, 1940s, naval battle, naval battleship, found family, godzilla, war orphan, survivor guilt, tokusatsu
Source: https://tickets.godzilla.com
Score: 1.0


Title: Ox-Head Village
Overview: Having launched a social media prank about a haunted building, three girls suddenly vanish. Rumours circulate that they were victims of The Ox-Head Village curse, triggering an investigation by two of their friends, desperate to find the truth about what has happened…
Release Date: 2022

In [5]:
print((response['output']))

- Dogman:
    - Runtime: 115 minutes
    - Release Date: September 27, 2023
    - Recommendation: "Dogman" is a heartfelt film that explores themes of love and salvation, similar to the emotional depth often found in Wes Anderson's movies. The bond between the boy and his dogs resonates with the quirky and touching relationships characteristic of Anderson's films.
    - Source: [Dogman Website](https://www.dogmanthefilm.com/)

- The Fabelmans:
    - Runtime: 151 minutes
    - Release Date: November 11, 2022
    - Recommendation: "The Fabelmans" delves into family dynamics and coming-of-age themes, reminiscent of Wes Anderson's storytelling style. The exploration of family secrets and the power of films to reveal truths aligns with the introspective and whimsical narratives often seen in Anderson's works.
    - Source: [The Fabelmans Website](https://www.thefabelmans.movie)


In [13]:
for chunk in agent_executor.stream({"input": "Recommend some films similar to Barbie."}):
    print(chunk, end="", flush=True)



> Entering new AgentExecutor chain...
{'actions': [OpenAIToolAgentAction(tool='movie_data', tool_input={'query': 'Barbie'}, log="\nInvoking: `movie_data` with `{'query': 'Barbie'}`\n\n\n", message_log=[AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UjYry3V8x5GwvxSL1ZnOq7FI', 'function': {'arguments': '{"query":"Barbie"}', 'name': 'movie_data'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})], tool_call_id='call_UjYry3V8x5GwvxSL1ZnOq7FI')], 'messages': [AIMessageChunk(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_UjYry3V8x5GwvxSL1ZnOq7FI', 'function': {'arguments': '{"query":"Barbie"}', 'name': 'movie_data'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'})]}
Invoking: `movie_data` with `{'query': 'Barbie'}`


Title: Barbie
Overview: Barbie and Ken are having the time of their lives in the colorful and seemingly perfect world of Barbie Land. However, when they get a chan

# Hybrid Approach v2

In [90]:
from langchain.chains import RetrievalQA

vector_qa = RetrievalQA.from_chain_type(
    llm=ChatOpenAI(model='gpt-3.5-turbo-0125',
                   temperature=0.5, streaming=True),
    chain_type="stuff",
    retriever=vector_store_window.as_retriever()
)

vector_qa.run(
    "Recommend me some Japanese horror films."
)

'Here are some Japanese horror films you might enjoy:\n\n1. **Ox-Head Village** (2022) - A film about missing persons and Japanese folklore directed by Takashi Shimizu.\n   \n2. **Lonely Castle in the Mirror** (2022) - A mysterious and enchanting film based on a novel, directed by Keiichi Hara.\n   \nThese films offer a mix of drama, mystery, and horror elements that might intrigue you based on your interest in Japanese horror.'

In [97]:
kg.refresh_schema()

cypher_chain = GraphCypherQAChain.from_llm(
    cypher_llm=ChatOpenAI(temperature=0, model_name='gpt-4-0125-preview'),
    qa_llm=ChatOpenAI(model='gpt-3.5-turbo-0125', temperature=0), graph=kg, verbose=True,
)

cypher_chain.run(
    "What are the titles of all of the A24 films you have?"
)



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Production_Company {name: 'A24'})-[:PRODUCED]->(f:Film) RETURN f.title
Full Context:
[{'f.title': 'Everything Everywhere All at Once'}, {'f.title': 'The Whale'}, {'f.title': 'X'}, {'f.title': 'The Zone of Interest'}, {'f.title': 'The Iron Claw'}, {'f.title': 'Dream Scenario'}, {'f.title': 'Past Lives'}]

> Finished chain.


'The titles of the A24 films are Everything Everywhere All at Once, The Whale, X, The Zone of Interest, The Iron Claw, Dream Scenario, and Past Lives.'

In [ ]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType

tools = [
    Tool(
        name="General Recommendations",
        func=vector_qa.run,
        description="""Useful when you need to recommend films that are 
        Not useful for counting the number of tasks.
        Use full question as input.
        """,
    ),
    Tool(
        name="Specific Recommedations",
        func=cypher_chain.run,
        description="""Useful when you need to answer questions about microservices,
        their dependencies or assigned people. Also useful for any sort of 
        aggregation like counting the number of tasks, etc.
        Use full question as input.
        """,
    ),
]

mrkl = initialize_agent(
    tools,
    ChatOpenAI(temperature=0, model_name='gpt-4-0125-preview'),
    agent=AgentType.OPENAI_FUNCTIONS, verbose=True
)

response = mrkl.run("Which team is assigned to maintain PaymentService?")
print(response)